# Evaluation of Extractive Summarization Algos

The goal is to research differences in the summarization efficiency of the popular summarization extractive algorithms with respect to the text language.

#### Languages:
- English
- Ukrainian
- Early New High German (?)

#### Algorithms:
- Random
- Frequency
- LexRank
- TextRank
- Luhn
- LSA (Latent Semantic Analysis)
- KL (Kullback-Leibler)

#### Evaluation approaches:
- SummaQA (https://github.com/ThomasScialom/summa-qa)
- BLANC (https://github.com/PrimerAI/blanc)
- SUPERT (https://github.com/yg211/acl20-ref-free-eval), too much?..
- BLEU (actually it's for MT...)  (**NOT SUITABLE - requires reference human-edited summary**)
- ROUGE (https://pypi.org/project/rouge-score/) (**NOT SUITABLE - requires reference human-edited summary**)


In [10]:
import requests
import json
import math
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.random import RandomSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [14]:
def get_request_url(lang):
    return f"https://datasets-server.huggingface.co/rows?dataset=wikimedia%2Fwikipedia&config=20231101.{lang}&split=train&offset=0&length={TEXT_COUNT}";


In [16]:
class LanguageConfig:
    def __init__(self, short_name, long_name):
        self.short = short_name
        self.long = long_name
        self.tokenizer = Tokenizer(long_name)
        self.stemmer = Stemmer(long_name)
        self.stop_words = get_stop_words(long_name)
        self.texts = self.get_texts(short_name)

    def get_texts(self, name):
        response = requests.get(get_request_url(name))
        return json.loads(response.text)

In [17]:
class Languages:
    def __init__(self, enLang, ukLang):
        self.en = enLang
        self.uk = ukLang

In [18]:
TEXT_COUNT = 10
SUMMARY_SIZE_PERCENTAGE = 5
languages = Languages(LanguageConfig('en', 'english'),
                      LanguageConfig('uk', 'ukrainian'))

### Generating summaries

In [19]:
class TextWithSummaries:
    def __init__(self, text, title, langConfig):
        stemmer = langConfig.stemmer
        tokenizer = langConfig.tokenizer
        stop_words = langConfig.stop_words
    
        self.text = text
        self.title = title
        self.random_summary = self.get_summary(RandomSummarizer(stemmer), tokenizer, stop_words)
        self.luhn_summary = self.get_summary(LuhnSummarizer(stemmer), tokenizer, stop_words)
        self.kl_summary = self.get_summary(KLSummarizer(stemmer), tokenizer, stop_words)
        self.reduction_summary = self.get_summary(ReductionSummarizer(stemmer), tokenizer, stop_words)
        self.text_rank_summary = self.get_summary(TextRankSummarizer(stemmer), tokenizer, stop_words)
        self.lex_rank_summary = self.get_summary(LexRankSummarizer(stemmer), tokenizer, stop_words)
        self.lsa_summary = self.get_summary(LsaSummarizer(stemmer), tokenizer, stop_words)
    
    def get_summary(self, summarizer, tokenizer, stop_words):
        parser = PlaintextParser.from_string(self.text, tokenizer)
        summarizer.stop_words = stop_words
        sentences_length = len(tokenizer.to_sentences(self.text))
        return summarizer(parser.document, math.floor(sentences_length * SUMMARY_SIZE_PERCENTAGE / 100))
        

In [99]:
text_with_summary = TextWithSummaries(languages.uk.texts['rows'][0]['row']['text'], languages.uk.texts['rows'][0]['row']['title'], languages.uk)

In [100]:
for sentence in text_with_summary.lsa_summary:
    print(sentence)

Складність об'єкта дослідження і широта предметної області зумовили диференціацію єдиної географії на низку спеціалізованих (галузевих) наукових дисциплін, які утворюють систему географічних наук
Природничо-географічні (фізико-географічні) науки вивчають закономірності будови й розвитку географічної оболонки та її окремих складових частин (рельєф, клімат, води, ґрунти, рослинний і тваринний світ)
Палеогеографія широко застосовує загальногеографічні методи й інструменти в історичному аспекті, виробивши на їхній основі власні спеціальні: палеогеоморфологічний, ~педологічний, ~кліматичний, ~ландшафтний та інші
Практичне завдання науки полягає у розробці палеогеографічних аспектів основ раціонального природокористування при пошуках і розвідці корисних копалин, меліорації земель, прогнозуванні майбутніх змін сучасного стану навколишнього середовища тощо
Розробка проблем перетворення природи в рамках стійкого розвитку людства задля найбільш ефективного використання природних ресурсів, що не 